<div style="font-size: 200%; font-weight: bold; color: maroon;">Machine Learning en SparkR - datos distribuidos <span style="color: green;">[solución]</span></div>
<!-- v. 1.2 - Paulo Villegas, 2020 -->

Adaptado de un [ejemplo de SparkR](https://github.com/apache/spark/blob/master/examples/src/main/r/ml/ml.R).

Vamos a ajustar un GLM para hacer una regresión de una variable contra dos predictores, uno numérico y otro categórico.

In [ ]:
library(SparkR)
library(ggplot2)

In [ ]:
spark <- sparkR.session( "local[*]" );

# Carga de datos

Usamos el típico [dataset de IRIS](https://en.wikipedia.org/wiki/Iris_flower_data_set). Está disponible por defecto en R; para trabajar con él en Spark lo convertimos en `SparkDataFrame`

In [ ]:
df <- as.DataFrame(iris)

head(df,5)

Separamos en conjunto de entrenamiento y test

In [ ]:
df.split = randomSplit( df, c(80,20))

# Ejercicio

## Interacción entre características de entrada

Hemos estimado la longitud de los sépalos en función de su anchura y de la especie de planta, pero considerando ambas características (<em>Sepal_Width</em> y <em>Species</em>) de forma separada. Sin embargo, es razonable esperar que ambas estén relacionadas: la especie influye en la anchura del pétalo (además de en su longitud). Por lo tanto, desde el punto de vista de la regresión, puede ser interesante considerar un término más que tenga en cuenta las _interacciones_ entre esas dos características.

Las fórmulas de R permiten definir esas interacciones fácilmente; SparkR soporta usar términos de interacción. 
Puede verse la sintaxis necesaria:
* en la [documentación sobre glm](https://stat.ethz.ch/R-manual/R-devel/library/stats/html/glm.html). 
* en la [documentación sobre fórmulas en R](https://stat.ethz.ch/R-manual/R-devel/library/stats/html/formula.html)


<span style="color: red;">Ejercicio:</span> *estime la longitud de los sépalos en función de 3 variables:* 
* `Sepal_Width`
* `Species` 
* y la interacción entre `Sepal_Width` y `Species`

<div class="alert alert-warning" role="alert" style="margin-top: 24pt; margin-bottom: 0pt;">Escriba el código necesario en la siguiente celda</div>

<br/>

<div class="alert alert-block alert-info">
Vamos a usar la versión con sintaxis SparkR, con <tt>spark.glm</tt><br/>
En realidad la llamada es la misma de antes; sólo cambia la fórmula que aplicamos, que al considerar la interacción entre <em>Sepal_Width</em> y <em>Species</em> tiene un término más (el de interacción)
</div>

In [ ]:
# Now fit a model using a new feature created based on interactions between the 2 base features.
gaussianGLM3 <- spark.glm(df.split[[1]], Sepal_Length ~ Sepal_Width + Species + Sepal_Width : Species, 
                          family = "gaussian")

In [ ]:
summary(gaussianGLM3)

In [ ]:
df.pred3 <- predict( gaussianGLM3, df.split[[2]])

In [ ]:
qplot( label, prediction, data=collect(df.pred3) ) + coord_fixed() + geom_smooth(method="lm")

<div class="alert alert-block alert-info">
Vamos a ver si hemos ganado algo en capacidad de predicción usando el término de interacción. Para ello calculamos el error cuadrático medio (RMSE) sin y con término de interacción, sobre el conjunto de test</div>

In [ ]:
rmse <- function( data ) 
{
    return (sqrt(mean((data$prediction-data$label)^2)))
}

In [ ]:
# Con interacciones:
rmse( collect(df.pred3) )

In [ ]:
# Sin interacciones:
gaussianGLM0 <- spark.glm(df.split[[1]], Sepal_Length ~ Sepal_Width + Species, family = "gaussian")
df.pred0 <- predict( gaussianGLM0, df.split[[2]])

rmse( collect(df.pred0) )

<div class="alert alert-block alert-info">
En este caso no hemos mejorado nada: los errores son prácticamente idénticos, de hecho en algunos casos (depende de la ejecución) resulta incluso ligeramente peor el caso que incluye interacciones (un síntoma de que el modelo puede estar sobreentrenando, lo cual es posible dado el conjunto de entrenamiento tan pequeño)
</div>

In [ ]:
# Veamos el efecto sobre el conjunto de entrenamiento
df.train1 = predict( gaussianGLM0, df.split[[1]])
df.train3 = predict( gaussianGLM3, df.split[[1]])

print( rmse( collect(df.train1) ))
print( rmse( collect(df.train3) ))